In [176]:
import requests
import pandas as pd
import random
import time
import datetime

In [205]:
# Retrieve API given client code
def get_auth_token():
    client_id = ""
    client_secret = ""
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=client_credentials&client_id={client_id}&client_secret={client_secret}"
    url = "https://accounts.spotify.com/api/token"

    response = requests.post(url=url, data=data, headers=headers)
    token = response.json()["access_token"]
    print(f"TOKEN {token} GENERATED")
    return {"Authorization": f"Bearer  {token}"}

auth = []
auth.append(get_auth_token())

KeyError: 'access_token'

In [128]:
def extract_artist(artist_json: dict):
    id = artist_json["id"]
    name = artist_json["name"]
    genres = artist_json["genres"]
    followers = artist_json["followers"]
    popularity = artist_json["popularity"]
    return {
        "id": id,
        "name": name,
        "genres": genres,
        "followers": followers,
        "popularity": popularity
        }

def get_artist(a_id: str, auth: list):
    artist_url = f"https://api.spotify.com/v1/artists/{a_id}"
    response = requests.get(url=artist_url, headers=auth[-1])

    # check for expired token
    if response.status_code == 401:
        print(f"TOKEN EXPIRED")
        auth.append(get_auth_token())
        return get_artist(a_id, auth)
    return response.json()

def get_related_artists(a_id: str, auth: list):
    related_url = f"https://api.spotify.com/v1/artists/{a_id}/related-artists"
    response = requests.get(url=related_url, headers=auth[-1])
    
    # check for expired token
    if response.status_code == 401:
        print(f"TOKEN EXPIRED")
        auth.append(get_auth_token())
        return get_related_artists(a_id, auth)

    return response.json()   

def extract_related_artists(related_json: dict, visited: set, sources: list):
    artists = []
    for a in related_json["artists"]:
        if a["id"] not in visited:
            visited.add(a["id"])
            sources.append(a["id"])
            artists.append(extract_artist(a))
    return artists

In [167]:
# initialization
# artist_dataset = pd.DataFrame(columns=["id", "name", "genres", "followers", "popularity"])
# visited_artists = set([]) # set of artists who are already in the dataset
# source_artists = [] # list of artists whose related artists have not been added

# reloading from saved point
artist_dataset = pd.read_csv("artist_dataset.csv")
visited_artists = set(artist_dataset['id'].to_list())
source_artists = artist_dataset['id'].to_list()[6000:]

In [95]:
# DEPRECATED, JUST FOR STARTING SEARCH

# starting_artists = []
# with open("starting_artists.txt", 'r') as f:
#     starting_artists = f.read().splitlines()

# for a in starting_artists:
#     artist_dataset.loc[len(artist_dataset)] = extract_artist(get_artist(a, auth))
#     visited_artists.add(a)
#     source_artists.append(a)

In [170]:
while len(visited_artists) < 75000 and len(source_artists) > 0:
    time.sleep(.5) # janky rate limiting
    search_id = source_artists.pop(random.randint(0, len(source_artists) - 1))
    related_artists = get_related_artists(search_id, auth)
    new_artists = pd.DataFrame(extract_related_artists(related_artists, visited_artists, source_artists))
    print(f"{len(new_artists)} ADDED")
    artist_dataset = pd.concat([artist_dataset, new_artists], ignore_index=True)

1 ADDED
10 ADDED
4 ADDED
14 ADDED
3 ADDED
1 ADDED
14 ADDED
10 ADDED
6 ADDED
11 ADDED
4 ADDED
2 ADDED
13 ADDED
6 ADDED
0 ADDED
8 ADDED
2 ADDED
10 ADDED
18 ADDED
6 ADDED
5 ADDED
4 ADDED
4 ADDED
6 ADDED
2 ADDED
14 ADDED
1 ADDED
9 ADDED
9 ADDED
0 ADDED
4 ADDED
3 ADDED
3 ADDED
11 ADDED
0 ADDED
10 ADDED
14 ADDED
14 ADDED
5 ADDED
13 ADDED
3 ADDED
5 ADDED
19 ADDED
10 ADDED
2 ADDED
14 ADDED
9 ADDED
4 ADDED
17 ADDED
2 ADDED
10 ADDED
11 ADDED
1 ADDED
5 ADDED
11 ADDED
1 ADDED
1 ADDED
19 ADDED
5 ADDED
11 ADDED
14 ADDED
0 ADDED
9 ADDED
1 ADDED
0 ADDED
4 ADDED
12 ADDED
0 ADDED
4 ADDED
0 ADDED
2 ADDED
12 ADDED
5 ADDED
6 ADDED
0 ADDED
13 ADDED
18 ADDED
5 ADDED
9 ADDED
13 ADDED
12 ADDED
9 ADDED
0 ADDED
6 ADDED
0 ADDED
0 ADDED
1 ADDED
9 ADDED
9 ADDED
11 ADDED
2 ADDED
3 ADDED
11 ADDED
2 ADDED
3 ADDED
1 ADDED
7 ADDED
0 ADDED
4 ADDED
3 ADDED
5 ADDED
4 ADDED
8 ADDED
7 ADDED
3 ADDED
6 ADDED
7 ADDED
11 ADDED
7 ADDED
3 ADDED
11 ADDED
10 ADDED
10 ADDED
2 ADDED
14 ADDED
0 ADDED
3 ADDED
12 ADDED
9 ADDED
10 ADDED
1

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [174]:
artist_dataset

,id,name,genres,followers,popularity
0,3wcj11K77LjEY1PkEazffa,Burna Boy,"['afrobeats', 'azontobeats', 'dancehall', 'nig...","{'href': None, 'total': 8517338}",79
1,6qqNVTkY8uBg9cP3Jd7DAH,Billie Eilish,"['art pop', 'pop']","{'href': None, 'total': 93029346}",88
2,4PzYKhC14sTJNEr0dzoo0d,Luísa Sonza,"['funk carioca', 'pop nacional']","{'href': None, 'total': 5423350}",76
3,3xvaSlT4xsyk6lY1ESOspO,Disney,['movie tunes'],"{'href': None, 'total': 488608}",81
4,0YC192cP3KPCRWx8zr8MfZ,Hans Zimmer,"['german soundtrack', 'orchestral soundtrack',...","{'href': None, 'total': 3817185}",79
...,...,...,...,...,...
51376,1nlVHZslyaaNB0p5SUJe17,Joe Mullins & The Radio Ramblers,"[banjo, bluegrass gospel]","{'href': None, 'total': 3255}",22
51377,435e39CJddwkCUEK86yIkg,Russell Moore & IIIrd Tyme Out,[bluegrass],"{'href': None, 'total': 4817}",30
51378,2a3OUnLZ0cSo8yu6JeXeYP,Newfound Road,"[banjo, bluegrass]","{'href': None, 'total': 5768}",28
51379,1DkccFev2GJdu3V9nnojdo,Ronnie Bowman,[bluegrass],"{'href': None, 'total': 5910}",33


In [180]:
# save data to csv
date = datetime.datetime.today().strftime('%Y-%m-%d')
artist_dataset.to_csv(f"artist_dataset-{date}.csv")

In [181]:
removal_list = []
def remove_artist(a_id):
    removal_list.append(a_id)    

In [197]:
def get_top_tracks(a_id: str, auth: list):
    url = f"https://api.spotify.com/v1/artists/{a_id}/top-tracks"
    response = requests.get(url, headers=auth[-1])
    
    if response.status_code == 401:
        print(f"TOKEN EXPIRED")
        auth.append(get_auth_token())
        return get_related_artists(a_id, auth[-1])

    # rate limit with exponential backoff
    sleep = 1
    while response.status_code == 429:
        print(f"Rate limited. Sleeping for {sleep} seconds.")
        time.sleep(sleep)
        response = requests.get(url, headers=auth[-1])
        if sleep < 250:
            sleep *= 2

    if response.status_code == 404:
        raise LookupError('Track not found')
    
    return response.json()

def extract_top_tracks(tracks_json):
    tracks = []
    print(tracks_json)
    for track in tracks_json["tracks"]:
        artists = track["artists"]
        duration = track["duration_ms"]
        explicit = track["explicit"]
        id = track["id"]
        name = track["name"]
        popularity = track["popularity"]
        row = {
            "id": id,
            "name": name,
            "artists": artists,
            "duration": duration,
            "explicit": explicit,
            "popularity": popularity
        }
        tracks.append(row)
    return tracks

In [202]:
song_dataset = pd.DataFrame(columns=['id', 'name', 'artists', 'duration', 'explicit', 'popularity'])

In [204]:
for a_id in artist_dataset['id']:
    try:
        time.sleep(.5)
        t = get_top_tracks(a_id, auth)
        new_songs = pd.DataFrame(extract_top_tracks(t))
        print(f"{len(new_songs)} songs ADDED for artist: {a_id}")
        artist_dataset = pd.concat([song_dataset, new_songs], ignore_index=True)
    except LookupError as e:
        print(f"No songs found for artist: {a_id}, adding to list.")
        remove_artist(a_id)
    

Rate limited. Sleeping for 1 seconds.
Rate limited. Sleeping for 2 seconds.
Rate limited. Sleeping for 4 seconds.
Rate limited. Sleeping for 8 seconds.
Rate limited. Sleeping for 16 seconds.
Rate limited. Sleeping for 32 seconds.
Rate limited. Sleeping for 64 seconds.
Rate limited. Sleeping for 128 seconds.
Rate limited. Sleeping for 256 seconds.
Rate limited. Sleeping for 256 seconds.


KeyboardInterrupt: 

In [161]:
artist_dataset

,id,name,artists,duration,explicit,popularity
0,22bZVqJfATps2ETj7017Ms,Spicy Margarita,[{'external_urls': {'spotify': 'https://open.s...,154376,False,73
1,1AM8QdDFZMq6SrrqUnuQ9P,Feeling Good,[{'external_urls': {'spotify': 'https://open.s...,237333,False,74
2,2ajUl8lBLAXOXNpG4NEPMz,Sway,[{'external_urls': {'spotify': 'https://open.s...,188066,False,73
3,3lyLqIn8mybyEFTs8JJaLf,Home,[{'external_urls': {'spotify': 'https://open.s...,225906,False,69
4,4fIWvT19w9PR0VVBuPYpWA,Haven't Met You Yet,[{'external_urls': {'spotify': 'https://open.s...,244586,False,70
5,4T6HLdP6OcAtqC6tGnQelG,Everything,[{'external_urls': {'spotify': 'https://open.s...,212373,False,70
6,0mvkwaZMP2gAy2ApQLtZRv,It's a Beautiful Day,[{'external_urls': {'spotify': 'https://open.s...,199266,False,65
7,1rv46mRwDqMEhOBZ7vODg3,It's Beginning to Look a Lot like Christmas,[{'external_urls': {'spotify': 'https://open.s...,207199,False,56
8,4m4aQLiPbWFyE07xRoJPhc,L O V E,[{'external_urls': {'spotify': 'https://open.s...,170213,False,60
9,67mgz7S5y7hnCE63YBjfO6,Holly Jolly Christmas,[{'external_urls': {'spotify': 'https://open.s...,121526,False,54
